In [3]:
import sys

!{sys.executable} -m pip install pandas

!{sys.executable} -m pip install bs4

!{sys.executable} -m pip install openpyxl

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, time, datetime, re, random

In [5]:
wiki_url = "https://vgsales.fandom.com/wiki/The_Legend_of_Zelda"
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
response = requests.get(wiki_url, headers=headers)
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

html_table = soup.find('table',{'class':"wikitable"})
html_table

switch_sales_df = pd.read_html(str(html_table))[0]
switch_sales_df

In [82]:
def copies_sold_to_numeric(value):
    parsed_string = re.sub(r'\*+','', value)
    return float(parsed_string) * 1000000

def convert_date(date):
     numeric_value = pd.to_numeric(date, errors='coerce')
     if pd.isna(numeric_value):
        return '' 
     else:
        return int(numeric_value)
     
def create_ign_url(title, release_date):
    if release_date != '':
        mod_title = re.sub(r'[.:,\'\*]',"", title)
        mod_title = mod_title.replace(" ", "-")
        return 'https://www.ign.com/games/' + mod_title.lower()
    else:
        return ''

In [83]:
switch_sales_df_formatted = switch_sales_df.assign(
    Release_date = lambda df: df['Year'].map(lambda var: convert_date(var)),
    Copies_sold = lambda df: df['Sales (millions)'].map(lambda var: copies_sold_to_numeric(var)),
    Ign_url=lambda df: df.apply(lambda row: create_ign_url(row['Title'], row['Release_date']), axis=1),
).drop(columns=['Year','Sales (millions)', 'Ref'], axis = 1)
switch_sales_df_formatted

,Platform,Title,Release_date,Copies_sold,Ign_url
0,NaN,The Legend of Zelda,,7290000.0,
1,NES,The Legend of Zelda,1986,6510000.0,https://www.ign.com/games/the-legend-of-zelda
2,GBA,Classic NES Series: The Legend of Zelda,2004,780000.0,https://www.ign.com/games/classic-nes-series-t...
3,NaN,Zelda II: The Adventure of Link,,4971000.0,
4,NES,Zelda II: The Adventure of Link,1987,4380000.0,https://www.ign.com/games/zelda-ii-the-adventu...
5,GBA,Classic NES Series: Zelda II: The Adventure of...,2004,591000.0,https://www.ign.com/games/classic-nes-series-z...
6,NaN,The Legend of Zelda: A Link to the Past,,7430000.0,
7,SNES,The Legend of Zelda: A Link to the Past,1992,4610000.0,https://www.ign.com/games/the-legend-of-zelda-...
8,GBA,The Legend of Zelda: A Link to the Past & Four...,2002,2820000.0,https://www.ign.com/games/the-legend-of-zelda-...
9,NaN,The Legend of Zelda: Link's Awakening,,12510000.0,


In [107]:
req = requests.get("https://www.ign.com/games/the-legend-of-zelda-breath-of-the-wild", headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

description = soup.find("div", class_ = "object-summary-text summary-info").text
#ign_rating = soup.find("figure", class_ = "review-score hexagon-wrapper").find("span", class_ = "hexagon-content-wrapper").find("figcaption").text
user_rating = soup.find("div", class_ = "stack jsx-2736506000 score-block high small").find("h3", class_="title5 jsx-1096646909").text
user_rate_num = soup.select_one("#main-content > div > div.jsx-3636063303.wave-header.object-page-heading.collapse > section > div > div.stack.jsx-1500469411.object-header > div.stack.jsx-774472442.meta-items.jsx-4085963266.alt.jsx-4284520940.ur-analytics-block > div:nth-child(3) > div.caption.jsx-1541923331.small.underlined").text
#user_rate_num = re.findall('[0-9]+', user_rate_num)[0]

user_rate_num


'7.7k Ratings'

In [122]:
def scrape_ign(html):
    soup = BeautifulSoup(html, 'html.parser')

    description_element = soup.find("div", class_="object-summary-text summary-info")
    description = description_element.text if description_element else "N/A"

    ign_rating_element = soup.select_one("#main-content > div > div.jsx-3636063303.wave-header.object-page-heading.collapse > section > div > div.stack.jsx-1500469411.object-header > div.stack.jsx-774472442.meta-items.jsx-4085963266.alt.jsx-4284520940.ur-analytics-block > div.stack.jsx-2736506000.analytic-with-text-box.ign-rating > figure > div > span > figcaption")
    # Assuming the content is always an integer
    ign_rating = ign_rating_element.text if ign_rating_element else ''


    user_rating_element = soup.find("div", class_="stack jsx-2736506000 score-block high small").find("h3", class_="title5 jsx-1096646909")
    user_rating = user_rating_element.text if user_rating_element else "N/A"

    user_rate_num_element = soup.select_one("#main-content > div > div.jsx-3636063303.wave-header.object-page-heading.collapse > section > div > div.stack.jsx-1500469411.object-header > div.stack.jsx-774472442.meta-items.jsx-4085963266.alt.jsx-4284520940.ur-analytics-block > div:nth-child(3) > div.caption.jsx-1541923331.small.underlined")
    user_rate_num_text = user_rate_num_element.text if user_rate_num_element else "N/A"

    # Extract numbers and multiplier (e.g., '7.7k' -> ('7.7', 'k'))
    matches = re.match(r'(\d*\.?\d+)([kKmM]*)', user_rate_num_text)
    if matches:
        value, multiplier = matches.groups()
        multiplier_dict = {'k': 1000, 'm': 1000000}
        multiplier_value = multiplier_dict.get(multiplier.lower(), 1)
        user_rate_num = int(float(value) * multiplier_value)
    else:
        user_rate_num = "N/A"

    return {
        'description': description,
        'ign_rating': ign_rating,
        'user_rating': float(user_rating) if user_rating.replace('.', '').isdigit() else "N/A",
        'user_rate_num': user_rate_num
    }

def perform_random_delay(delay = 3, random_offset = 1):
    time.sleep(delay + random.uniform(0,random_offset))

In [123]:
urls = switch_sales_df_formatted["Ign_url"].tolist()
ign_results = []

for url in urls:
    if url:
        req = requests.get(url, headers=headers)
        if req.status_code==200:
            try:
                # attempt to parse page
                curr_url_result = scrape_ign(req.text)
                curr_url_result['Ign_url'], curr_url_result['status_code'] = url, 200
                curr_url_result['parsed'] = True
            except:
                # tell user site reached but parse error
                curr_url_result = {'description' : '', 'ign_rating' : '', 'user_rating' : '', 
                            'user_rate_num' : '', 'status_code' : 200, 'Ign_url' : url, 'parsed' : False}
        else:
            # page cannot be reached
            curr_url_result = {'description' : '', 'ign_rating' : '', 'user_rating' : '', 
                            'user_rate_num' : '', 'status_code' : req.status_code, 'Ign_url' : url, 'parsed' : False}
        # append results
        ign_results.append(curr_url_result)
        # perform random relay between scrapes
        perform_random_delay()

In [124]:
reviews_df = pd.DataFrame(ign_results)
reviews_df

,description,ign_rating,user_rating,user_rate_num,Ign_url,status_code,parsed
0,You must defeat a variety of enemies as you se...,9,8.2,542,https://www.ign.com/games/the-legend-of-zelda,200,True
1,,,,,https://www.ign.com/games/classic-nes-series-t...,404,False
2,Link returns to Hyrule to search for the Trifo...,8.5,7.0,210,https://www.ign.com/games/zelda-ii-the-adventu...,200,True
3,,,,,https://www.ign.com/games/classic-nes-series-z...,404,False
4,When an evil magician named Agahnim begins kid...,9.5,9.2,1100,https://www.ign.com/games/the-legend-of-zelda-...,200,True
5,,,,,https://www.ign.com/games/the-legend-of-zelda-...,404,False
6,The Legend of Zelda: Link's Awakening is a gra...,9.4,8.4,1400,https://www.ign.com/games/the-legend-of-zelda-...,200,True
7,Nintendo upgraded its Game Boy hit for the GBC...,10,8.8,223,https://www.ign.com/games/the-legend-of-zelda-...,200,True
8,The Legend of Zelda: Link's Awakening is a gra...,9.4,8.4,1400,https://www.ign.com/games/the-legend-of-zelda-...,200,True
9,Journey through the land of Hyrule in the Nint...,10,9.6,1700,https://www.ign.com/games/the-legend-of-zelda-...,200,True


In [125]:
switch_sales_reviews_df = switch_sales_df_formatted.merge(reviews_df)
switch_sales_reviews_df

,Platform,Title,Release_date,Copies_sold,Ign_url,description,ign_rating,user_rating,user_rate_num,status_code,parsed
0,NES,The Legend of Zelda,1986,6510000.0,https://www.ign.com/games/the-legend-of-zelda,You must defeat a variety of enemies as you se...,9,8.2,542,200,True
1,GBA,Classic NES Series: The Legend of Zelda,2004,780000.0,https://www.ign.com/games/classic-nes-series-t...,,,,,404,False
2,NES,Zelda II: The Adventure of Link,1987,4380000.0,https://www.ign.com/games/zelda-ii-the-adventu...,Link returns to Hyrule to search for the Trifo...,8.5,7.0,210,200,True
3,GBA,Classic NES Series: Zelda II: The Adventure of...,2004,591000.0,https://www.ign.com/games/classic-nes-series-z...,,,,,404,False
4,SNES,The Legend of Zelda: A Link to the Past,1992,4610000.0,https://www.ign.com/games/the-legend-of-zelda-...,When an evil magician named Agahnim begins kid...,9.5,9.2,1100,200,True
5,GBA,The Legend of Zelda: A Link to the Past & Four...,2002,2820000.0,https://www.ign.com/games/the-legend-of-zelda-...,,,,,404,False
6,GB,The Legend of Zelda: Link's Awakening,1993,3830000.0,https://www.ign.com/games/the-legend-of-zelda-...,The Legend of Zelda: Link's Awakening is a gra...,9.4,8.4,1400,200,True
7,GB,The Legend of Zelda: Link's Awakening,1993,3830000.0,https://www.ign.com/games/the-legend-of-zelda-...,The Legend of Zelda: Link's Awakening is a gra...,9.4,8.4,1400,200,True
8,GBC,The Legend of Zelda: Link's Awakening DX,1998,2220000.0,https://www.ign.com/games/the-legend-of-zelda-...,Nintendo upgraded its Game Boy hit for the GBC...,10,8.8,223,200,True
9,Switch,The Legend of Zelda: Link's Awakening,2019,6460000.0,https://www.ign.com/games/the-legend-of-zelda-...,The Legend of Zelda: Link's Awakening is a gra...,9.4,8.4,1400,200,True


In [126]:
sum(switch_sales_reviews_df['parsed']) / switch_sales_reviews_df.shape[0]

0.8787878787878788

In [128]:
switch_sales_reviews_df.to_csv("Zelda Sales and Reviews.csv", index = False)